## load lib

In [130]:
import pandas as pd
from pandas import DataFrame
import csv
import numpy as np

## load file for attributes

In [278]:
attribute_file = "./source/lib/attribute.csv"
f = open(attribute_file, "r", encoding = "utf-8")
data = csv.reader(f)
f.close
next(data)
attributes = []
for row in data:
    if row[-1]:
        attributes.append(row[0])

In [279]:
attributes

['짜다',
 '저렴하다',
 '맛있다',
 '맵다',
 '느끼하다',
 '유명하다',
 '달다',
 '고소하다',
 '신선하다',
 '고급스럽다',
 '뜨겁다']

## save root

In [322]:
from gensim.models.word2vec import Word2Vec

criteria_brand = ["삼양라면"]
criteria_attributes = []

root_model = "./source/model/word2vec_라면_total.model"
model = Word2Vec.load(root_model)

for attribute in attributes:
    if model.wv.similarity(criteria_brand, attribute)[0] > 0.5:
        criteria_attributes.append(attribute)
        
coeff = pd.DataFrame()
coeff = pd.DataFrame(coeff, columns = criteria_attributes, index = criteria_brand)
for c_attribute in criteria_attributes:
    buff = float(np.round(((model.wv.similarity(criteria_brand, c_attribute) + 1) / 2), 5))
    coeff[c_attribute] = buff
    


In [323]:
coeff

,저렴하다,맵다,느끼하다,유명하다,달다,고소하다,고급스럽다
삼양라면,0.79225,0.80365,0.77332,0.85229,0.78347,0.76139,0.83765


## load models for comparison 

In [213]:
comparison_file = "./source/lib/brands.csv"
f = open(comparison_file, "r", encoding = "utf-8")
data = csv.reader(f)
f.close
next(data)
brands = []
for row in data:
    # 브랜드라면 brands에 추가
    if row[-1]:
        brands.append(row[0])

comp_model_file ="./source/model/word2vec_치킨_total.model"
comp_model = Word2Vec.load(comp_model_file)


In [214]:
comp_model.wv.most_similar("호식이")

[('식', 0.7238513231277466),
 ('식이', 0.6695153713226318),
 ('젠', 0.6276451349258423),
 ('뿐', 0.6236096620559692),
 ('나라', 0.6216815710067749),
 ('순삭', 0.6215479969978333),
 ('꼬식', 0.6209991574287415),
 ('비다', 0.620478630065918),
 ('모자라다', 0.6179180145263672),
 ('편', 0.6175099015235901)]

## calculate rating
### 설명:
### model.vw.similarity() 는 -1 (극단적으로 멀다) 과 1 (극단적으로 가깝다)사이의 값이 나옴.
### 그 둘을 확률처럼 변형하고 곱한 것이 rating
### total rating은 10점 만점으로 나타냄

In [324]:
rating = pd.DataFrame()
rating = pd.DataFrame(rating, columns = criteria_attributes, index = brands)

# attribute와 brands의 similarity와 coeff의 곱을 rating에 저장
for c_attribute in criteria_attributes:
    for brand in brands:
        rating.loc[brand, c_attribute] = round((((comp_model.wv.similarity(brand, c_attribute) + 1) / 2 ) * coeff.loc[criteria_brand[0], c_attribute]), 5)
#total of columns 추가
rating["total"] = (rating.sum(axis = 1))/len(criteria_attributes) * 10

In [327]:
display(rating)

,저렴하다,맵다,느끼하다,유명하다,달다,고소하다,고급스럽다,total
뿌링클,0.64025,0.58937,0.59680,0.70541,0.61997,0.57791,0.65422,6.262757
레드콤보,0.61001,0.59740,0.60170,0.69099,0.59836,0.58423,0.63219,6.164114
황금올리브,0.62173,0.58017,0.54918,0.68103,0.55748,0.55722,0.63781,5.978029
푸라닭,0.62602,0.61060,0.52408,0.69793,0.57575,0.53366,0.62905,5.995843
호식이,0.58516,0.57236,0.55828,0.65595,0.58565,0.51334,0.62428,5.850029


In [139]:
rating.idxmax()

짜다        짜파게티
저렴하다     불닭볶음면
맛있다       짜파게티
맵다       불닭볶음면
느끼하다     불닭볶음면
다이어트      짜파게티
달다         진라면
유명하다       진라면
비싸다        신라면
total      진라면
dtype: object